# Import libraries

In [21]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

from tqdm.notebook import tqdm

# Loading data

In [2]:
#Прочитаем данные
locations = pd.read_csv('locations.csv', sep=',')

print(locations.info())
locations.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7812 entries, 0 to 7811
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       7812 non-null   float64
 1   y       7812 non-null   float64
dtypes: float64(2)
memory usage: 122.2 KB
None


,x,y
0,389.0,-770.0
1,338.0,-844.0
2,455.0,-1227.0
3,408.0,-1119.0
4,334.0,-1121.0


# Finding the optimal number of clusters

In [ ]:
def distance(x1, x2, y1, y2):   
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [3]:
# Calculating the clustering quality metric
metrics = []
MAX_CLUSTERS = 1000

for num_cluster in range(1, MAX_CLUSTERS + 1):
    kmeans_model = KMeans(n_clusters=num_cluster, random_state=42).fit(locations)
    centroids, labels = kmeans_model.cluster_centers_, kmeans_model.labels_
    metric = 0
    
    for cluster_label in range(num_cluster):
        metric += euclidean_distances(
            locations[labels==cluster_label],
            centroids[cluster_label, :].reshape(1, -1)
        ).sum(axis=0)[0]
        
        
    if num_cluster % 100 == 0:
        print("For the number of clusters %s, the clustering quality metric is %s" % (num_cluster, metric))
        
    metrics.append(metric)

Для количества кластеров 100 метрика качества кластеризации равна 135536.0345908121
Для количества кластеров 200 метрика качества кластеризации равна 83578.50270539003
Для количества кластеров 300 метрика качества кластеризации равна 63160.41832941112
Для количества кластеров 400 метрика качества кластеризации равна 50801.072113797316
Для количества кластеров 500 метрика качества кластеризации равна 43230.989776511495
Для количества кластеров 600 метрика качества кластеризации равна 37175.68178128163
Для количества кластеров 700 метрика качества кластеризации равна 33011.07455645412
Для количества кластеров 800 метрика качества кластеризации равна 29733.245121361804
Для количества кластеров 900 метрика качества кластеризации равна 26617.953689952043
Для количества кластеров 1000 метрика качества кластеризации равна 24318.940014913234


In [43]:
# The elbow method
import matplotlib.pyplot as plt

# Analytical solution
D = []
for i in range(0, len(metrics) - 1):
    d = np.abs(metrics[i+1] - metrics[i])/np.abs(metrics[i] - metrics[i-1])
    D.append(d)
    
print("(Analytically) The best number of clusters is %s" % (np.argmin(D) + 1))

(Аналитически) Лучшее количество кластеров равно 997


In [6]:
kmeans = KMeans(n_clusters=997)
kmeans.fit(locations)

KMeans(n_clusters=997)

### Расчет положения стендов

In [22]:
dict_clusters = {}

clients = locations.values

for center in tqdm(kmeans.cluster_centers_):
    for ind, client in enumerate(clients):
        if distance(center[0], client[0], center[1], client[1]) <= 10:
            if (center[0], center[1]) not in dict_clusters.keys():
                dict_clusters[(center[0], center[1])] = 1
            else:
                dict_clusters[(center[0], center[1])] += 1
            
            np.delete(clients, ind)

  0%|          | 0/997 [00:00<?, ?it/s]

In [34]:
sorted_tuples = sorted(dict_clusters.items(), key=lambda item: item[1], reverse=True)
sorted_dict = {k: v for k, v in sorted_tuples}

In [42]:
stands = np.array([np.array(stand) for stand in list(sorted_dict.keys())[:15]])

answer = pd.DataFrame(stands)
answer.to_csv('answer.csv', index=False)